# Task 3.1: Analyzing Suspicious Audio Files for Hidden Data

## Objective
The aim is to analyze a set of suspicious audio files, identify which one contains hidden ultrasonic data, and shift the secret message to an audible range for playback. This is achieved using Fourier analysis to isolate ultrasonic frequencies.

## Implementation
The analysis involves:
1. Reading and converting audio files to numerical data.
2. Performing Fourier Transform to analyze the frequency domain.
3. Identifying ultrasonic frequencies and calculating their power.
4. Shifting ultrasonic data to an audible range.
5. Saving the processed audio for playback.

In [1]:
# Import necessary libraries
import numpy as np  # For numerical computations
import wave  # For audio file handling
import os  # For file and directory operations

In [2]:
# Function to analyze audio files
def analyze_audio_files(input_files, output_dir, ultrasonic_threshold=20000):
    """
    Analyze a group of audio files to detect ultrasonic frequencies and shift them to an audible range.

    Args:
        input_files (list): List of audio file paths to analyze.
        output_dir (str): Directory where processed files will be saved.
        ultrasonic_threshold (int): Frequency above which data is considered ultrasonic.

    Returns:
        str: The filename containing the highest ultrasonic power.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    detected_file = None  # To store the file with the highest ultrasonic power
    highest_power = 0  # Track the highest detected ultrasonic power
    log_file = os.path.join(output_dir, "analysis_log.txt")  # Log file for results

    # Open log file to record results
    with open(log_file, "w") as log:
        log.write("Audio Analysis Results:\n")
        log.write("File Name\tUltrasonic Power\n")

        # Analyze each audio file
        for file in input_files:
            print(f"Analyzing {file}...")
            with wave.open(file, 'rb') as infile:
                params = infile.getparams()  # Audio file parameters
                framerate = infile.getframerate()  # Sampling rate
                nframes = infile.getnframes()  # Number of frames
                audio_data = infile.readframes(nframes)  # Raw audio data
                
                # Convert raw audio data to a numpy array
                audio_signal = np.frombuffer(audio_data, dtype=np.int16)
                
                # Perform Fourier Transform to get frequency domain data
                freq_domain = np.fft.rfft(audio_signal)
                frequencies = np.fft.rfftfreq(len(audio_signal), d=1/framerate)
                
                # Calculate the power of ultrasonic frequencies
                ultrasonic_power = np.sum(np.abs(freq_domain[frequencies > ultrasonic_threshold]))
                log.write(f"{file}\t{ultrasonic_power}\n")
                print(f"Ultrasonic power in {file}: {ultrasonic_power}")

                # Update the file with the highest ultrasonic power
                if ultrasonic_power > highest_power:
                    highest_power = ultrasonic_power
                    detected_file = file
                
                # Shift ultrasonic frequencies to the audible range
                audible_signal = np.zeros_like(freq_domain)
                for i, freq in enumerate(frequencies):
                    if freq > ultrasonic_threshold:
                        new_index = i - int(ultrasonic_threshold * len(frequencies) / framerate)
                        if new_index >= 0:
                            audible_signal[new_index] = freq_domain[i]
                
                # Convert the shifted signal back to the time domain
                shifted_signal = np.fft.irfft(audible_signal)
                shifted_signal = np.int16(shifted_signal / np.max(np.abs(shifted_signal)) * 32767)
                
                # Save the processed audio file
                output_file = os.path.join(output_dir, os.path.basename(file))
                with wave.open(output_file, 'wb') as outfile:
                    outfile.setparams(params)
                    outfile.writeframes(shifted_signal.tobytes())
                
                print(f"Processed file saved as {output_file}")
        
        log.write(f"\nMost suspicious file: {detected_file}\n")
    print(f"Analysis log saved to {log_file}")
    return detected_file

In [3]:
# Example usage
input_files = ["Ex3_sound1.wav", "Ex3_sound2.wav", "Ex3_sound3.wav", "Ex3_sound4.wav"]
output_dir = "processed_files"
detected_file = analyze_audio_files(input_files, output_dir)

# Display results
if detected_file:
    print(f"Suspicious file detected: {detected_file}")
else:
    print("No suspicious ultrasonic data detected.")

Analyzing Ex3_sound1.wav...
Ultrasonic power in Ex3_sound1.wav: 4447888184.787553
Processed file saved as processed_files\Ex3_sound1.wav
Analyzing Ex3_sound2.wav...
Ultrasonic power in Ex3_sound2.wav: 13946298.077507427
Processed file saved as processed_files\Ex3_sound2.wav
Analyzing Ex3_sound3.wav...
Ultrasonic power in Ex3_sound3.wav: 12148842.54451987
Processed file saved as processed_files\Ex3_sound3.wav
Analyzing Ex3_sound4.wav...
Ultrasonic power in Ex3_sound4.wav: 6566647102.476889
Processed file saved as processed_files\Ex3_sound4.wav
Analysis log saved to processed_files\analysis_log.txt
Suspicious file detected: Ex3_sound4.wav
